In [29]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq
 
from bokeh.palettes import brewer, Spectral6
from bokeh.io import show, output_file, curdoc
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Panel, Tabs, GeoJSONDataSource, LinearColorMapper
from bokeh.models import ColorBar
from bokeh.layouts import column, gridplot, row
from bokeh.transform import factor_cmap
from bokeh.models import NumeralTickFormatter, Title, Label, Paragraph, Div, CustomJSHover, BoxAnnotation

This file creates the aggregate trade data.


In [30]:
trade_type = "imports"

my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

In [31]:
crl = ["darkblue","slategray","crimson"]

background = "#ffffff"

In [32]:
last_month = 8

---
#### Grabe the trade data using the Census's API

In [33]:
def census_trade(url, trade_type, country, product_level):
    
    r = requests.get(url) 
    
    print(r)
    
    df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    df.columns = r.json()[0]

    df.time = pd.to_datetime(df.time, format="%Y-%m")
    # This is so I can call this correctly...
    
    if trade_type == "imports":
        
        trade_label = country + "_" + trade_type
        
        df[trade_label] = df["CON_VAL_MO"].astype(float)
        
        df[product_level] = df["NAICS"].astype(str)
        
        df.drop(["CON_VAL_MO", "NAICS", "COMM_LVL"], axis = 1, inplace = True)
        
    if trade_type == "exports":
    
        trade_label = country + "_" + trade_type
        
        df[trade_label] = df["ALL_VAL_MO"].astype(float)

        df[product_level] = df["NAICS"].astype(str)
        
        df.drop(["ALL_VAL_MO", "NAICS", "COMM_LVL"], axis = 1, inplace = True)
        
    if trade_type == "country_exports":
    
        trade_label = "total_exports"
        
        df[trade_label] = df["ALL_VAL_MO"].astype(float)
        
        df = df[df.SUMMARY_LVL == "DET"]
        
        df.drop(["ALL_VAL_MO", "CTY_CODE", "SUMMARY_LVL"], axis = 1, inplace = True)
        
        df = df[df.CTY_NAME != "TOTAL FOR ALL COUNTRIES"]
        
    if trade_type == "country_imports":
    
        trade_label = "total_imports"
        
        df[trade_label] = df["CON_VAL_MO"].astype(float)
        
        df = df[df.SUMMARY_LVL == "DET"]
        
        df.drop(["CON_VAL_MO", "CTY_CODE", "SUMMARY_LVL"], axis = 1, inplace = True)
        
        
    
    return df

---
### And this is the Product Plot

In [34]:
end_use = "naics?get=NAICS,CON_VAL_MO,NAICS_LDESC"

url = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 
url = url + my_key + "&time==from+2013-01" + "&COMM_LVL=NA6"

url = url

df = census_trade(url, trade_type, "total", "naics")

df.time.max()

<Response [200]>


Timestamp('2020-06-01 00:00:00')

In [35]:
df.head()

,NAICS_LDESC,time,total_imports,naics
0,OPHTHALMIC GOODS,2013-01-01,4.007851e+08,339115
1,DENTAL LABORATORIES PRODUCTS,2013-01-01,2.837430e+07,339116
2,JEWELRY AND SILVERWARE,2013-01-01,3.134954e+09,339910
3,SPORTING AND ATHLETIC GOODS,2013-01-01,6.765605e+08,339920
4,"DOLLS, TOYS, AND GAMES",2013-01-01,1.293243e+09,339930


In [36]:
#end_use = "hs?get=E_COMMODITY,ALL_VAL_MO,E_COMMODITY_SDESC"

#url = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 
#url = url + my_key + "&time==from+2013-01" + "&COMM_LVL=HS2"

#url = url

#df = census_trade(url, trade_type, "total", "hs2")

#df.time.max()

In [37]:
def growth_trade(foo):
    # what this function does is take a dataframe and create a relative 
    
    foo["growth"] = 100*((foo.total_imports/foo.total_imports.shift(12)) - 1)
    
    return foo

In [38]:
def make_trade_time(df):
        
    foo = df.drop(labels = ["naics"], axis = 1).reset_index().pivot(index = "time", columns = "NAICS_LDESC")
    
    foo.drop(labels = ["index"], axis = 1, inplace = True)
    
    return foo

In [39]:
grp = df.groupby(["naics"])

#top_products = grp.agg({"total_imports":"sum"})

#plist = list(top_products.sort_values(by = "total_imports", ascending = False).index.values)[0:50]

#df = df[df.naics.isin(plist)]

In [40]:
grp = df.groupby(["naics"])

productdf = grp.apply(growth_trade)

In [41]:
productdf[productdf["time"] > "2020"].sort_values(by = ["growth"], ascending = False).head(50)

,NAICS_LDESC,time,total_imports,naics,growth
35241,SUGAR BEETS,2020-04-01,2.381000e+04,111991,inf
35978,GUIDED MISSILES AND SPACE VEHICLES,2020-05-01,8.200000e+06,336414,inf
35222,RICE,2020-04-01,2.143000e+03,111160,inf
35266,GOLD ORES,2020-04-01,1.323590e+05,212221,inf
34464,GOLD ORES,2020-02-01,9.980130e+06,212221,1.327912e+05
34837,COTTON,2020-03-01,1.974758e+06,111920,4.249400e+03
36068,GOLD ORES,2020-06-01,6.070570e+05,212221,3.430017e+03
34462,"ANTHRACITE COAL, NOT AGGLOMERATED",2020-02-01,1.214020e+06,212113,3.258006e+03
34838,SUGARCANE,2020-03-01,5.249300e+05,111930,3.147927e+03
35264,"ANTHRACITE COAL, NOT AGGLOMERATED",2020-04-01,1.366013e+06,212113,2.627444e+03


In [42]:
def make_time_by_product(df, level):

    height = 533
    width = 625

    foobar = make_trade_time(df)
    
    if level == "total_imports":
        title = "By Product (NAICS 3), $ US Imports "
    
    if level == "growth":
        title = "By Product (NAICS 3), % Year over Year Change in US Imports"
    
    p = figure(plot_height=height, plot_width = width, x_axis_type="datetime",toolbar_location = 'below',
           tools = "box_zoom, reset, pan, xwheel_zoom", title = title, x_range = (dt.datetime(2019,1,1),dt.datetime(2020,last_month,1)) ) 

    numlines=len(foobar[level].columns)

    multi_line_source = ColumnDataSource({
        'xs': [foobar.index.values]*numlines,
        'ys': [foobar[level, name].values for name in foobar[level]],
        'label': [name for name in foobar["total_imports"].columns]})

    p.multi_line(xs= "xs",
                ys= "ys",
                line_width=2.5, line_alpha=0.45, line_color = "slategray",
                 hover_line_alpha=0.75, hover_line_width = 5,
                hover_line_color= "crimson", source = multi_line_source)
    
    y_custom = CustomJSHover(code=""" return '' + special_vars.data_y
            """)
            
    TIMETOOLTIPS = """
            <div style="background-color:#F5F5F5; opacity: 0.95; border: 5px 5px 5px 5px;">
            <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold"> @label
             </span>
             </div>
             <div style = "text-align:left;">"""
    
    if level == "total_imports":
        TIMETOOLTIPS = TIMETOOLTIPS + """
            <span style="font-size: 13px; font-weight: bold"> $data_x{%b %Y}:  $data_y{$0.0a}</span>   
            </div>
            </div>
            """
    if level == "growth":
        TIMETOOLTIPS = TIMETOOLTIPS + """
            <span style="font-size: 13px; font-weight: bold"> $data_x{%b %Y}:  $data_y{0.0}%</span>   
            </div>
            </div>
            """

    p.add_tools(HoverTool(tooltips = TIMETOOLTIPS,  line_policy='nearest', formatters={'$data_x': 'datetime'}))
    p.title.text_font_size = '13pt'
    p.background_fill_color = background 
    p.background_fill_alpha = 0.75
    p.border_fill_color = background 
    
    #tradewar_box = BoxAnnotation(left=dt.datetime(2018,7,1), right=dt.datetime(2019,10,11), fill_color='red', fill_alpha=0.1)
    #p.add_layout(tradewar_box)
    
    pandemic_box = BoxAnnotation(left=dt.datetime(2020,3,1), right=dt.datetime(2020,12,31), fill_color='red', fill_alpha=0.1)
    p.add_layout(pandemic_box)
    
    #p.yaxis.axis_label = 
    p.yaxis.axis_label_text_font_style = 'bold'
    p.yaxis.axis_label_text_font_size = "13px"

    p.yaxis.minor_tick_line_color = None
    #p.y_range.start = 0
    
    #p.x_range.max_interval = dt.datetime(2020,6,1)
    
    #description = """Each line is a major product catagory in Annex 6-1 of The Agreement."""
    #p.add_layout(Title(text=description, text_font_style="normal", text_font_size="9pt"), 'below')
    #description = "Red marks the period of Section 301 tariffs and retaliation. Blue is period of agreement."""
    #p.add_layout(Title(text=description, text_font_style="normal", text_font_size="9pt"), 'below')
    
    div0 = Div(text = """Each line is a 3 digit NAICS catagory. Only the top 10 catagories are shown. Red is the start of the NBER
    determined recession and Covid-19 pandemic in the US.""", width=555, background = background )
    
    if level == "total_imports":
        p.yaxis.formatter = NumeralTickFormatter(format="($0. a)")
    
    p.outline_line_color = None
    
    p.toolbar.active_drag = None
    p.toolbar.autohide = True
    
    p.sizing_mode= "scale_both"
    p.max_height = height
    p.max_width = width
    p.min_border_left = 0
    p.min_border_bottom = 0
    
    return p

---
### This is the Country plot...

In [43]:
end_use = "naics?get=NAICS,CON_VAL_MO,CTY_CODE,CTY_NAME,SUMMARY_LVL,NAICS_LDESC"

url = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 
url = url + my_key + "&time==from+2013-01" + "&COMM_LVL=NA4" + "&NAICS=3313"

#end_use = "naics?get=NAICS,CON_VAL_MO,NAICS_LDESC"

#url = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 
#url = url + my_key + "&time==from+2013-01" + "&COMM_LVL=NA6"


url = url

df = census_trade(url,"country_imports","foo", "foo")

df.time.max()

<Response [200]>


Timestamp('2020-06-01 00:00:00')

In [44]:
df.head()

,NAICS,CTY_NAME,NAICS_LDESC,time,COMM_LVL,NAICS,total_imports
0,3313,TOTAL FOR ALL COUNTRIES,ALUMINA AND ALUMINUM AND PROCESSING,2013-01-01,NA4,3313,9.852413e+08
1,3313,TOTAL FOR ALL COUNTRIES,ALUMINA AND ALUMINUM AND PROCESSING,2013-02-01,NA4,3313,8.859865e+08
2,3313,TOTAL FOR ALL COUNTRIES,ALUMINA AND ALUMINUM AND PROCESSING,2013-03-01,NA4,3313,1.056654e+09
3,3313,TOTAL FOR ALL COUNTRIES,ALUMINA AND ALUMINUM AND PROCESSING,2013-04-01,NA4,3313,1.067720e+09
4,3313,TOTAL FOR ALL COUNTRIES,ALUMINA AND ALUMINUM AND PROCESSING,2013-05-01,NA4,3313,1.100316e+09


In [45]:
df = df[["CTY_NAME","time","total_imports"]]

In [46]:
df.head()

,CTY_NAME,time,total_imports
0,TOTAL FOR ALL COUNTRIES,2013-01-01,9.852413e+08
1,TOTAL FOR ALL COUNTRIES,2013-02-01,8.859865e+08
2,TOTAL FOR ALL COUNTRIES,2013-03-01,1.056654e+09
3,TOTAL FOR ALL COUNTRIES,2013-04-01,1.067720e+09
4,TOTAL FOR ALL COUNTRIES,2013-05-01,1.100316e+09


In [47]:
grp = df.groupby(["CTY_NAME"])

top_products = grp.agg({"total_imports":"sum"})

clist = list(top_products.sort_values(by = "total_imports", ascending = False).index.values)[0:5]

df = df[df.CTY_NAME.isin(clist)]

In [48]:
grp = df.groupby(["CTY_NAME"])

country_df = grp.apply(growth_trade)

In [49]:
def make_trade_time_country(df):
        
    foo = df.reset_index().pivot(index = "time", columns = "CTY_NAME")
    
    foo.drop(labels = ["index"], axis = 1, inplace = True)
    
    return foo

In [50]:
test = make_trade_time_country(country_df)

In [51]:
test["total_imports"].drop([ "TOTAL FOR ALL COUNTRIES"], axis = 1)

CTY_NAME,CANADA,CHINA,RUSSIA,UNITED ARAB EMIRATES
time,,,,
2013-01-01,484239884.0,72801893.0,29293735.0,61699909.0
2013-02-01,429279652.0,51513082.0,49213198.0,58322359.0
2013-03-01,569882599.0,55376166.0,53613863.0,48515775.0
2013-04-01,594498932.0,71029639.0,53654732.0,57267314.0
2013-05-01,566488698.0,60615376.0,76173964.0,43592148.0
...,...,...,...,...
2020-02-01,430371584.0,33010067.0,57743429.0,57922619.0
2020-03-01,550127809.0,35319577.0,30167353.0,92776432.0
2020-04-01,482388469.0,40225815.0,36612128.0,74929115.0


In [52]:
def make_time_by_country(df, level):

    height = 533
    width = 625

    foobar = make_trade_time_country(df)
    
    if level == "total_imports":
        title = "By Source Country, $ US Imports (NAICS 3313, ALUMINUM)"
        
        foobar.drop([ "TOTAL FOR ALL COUNTRIES"], axis = 1, level = 1, inplace = True)
    
    if level == "growth":
        title = "By Source Country, % Year over Year Change in US Imports (NAICS 3313, ALUMINUM)"
    
    p = figure(plot_height=height, plot_width = width, x_axis_type="datetime",toolbar_location = 'below',
           tools = "box_zoom, reset, pan, xwheel_zoom", title = title, x_range = (dt.datetime(2017,1,1),dt.datetime(2020,last_month,1)) ) 

    numlines=len(foobar[level].columns)

    multi_line_source = ColumnDataSource({
        'xs': [foobar.index.values]*numlines,
        'ys': [foobar[level, name].values for name in foobar[level]],
        'label': [name for name in foobar["total_imports"].columns]})

    p.multi_line(xs= "xs",
                ys= "ys",
                line_width=2.5, line_alpha=0.45, line_color = "slategray",
                 hover_line_alpha=0.75, hover_line_width = 5,
                hover_line_color= "crimson", source = multi_line_source)
    
    y_custom = CustomJSHover(code=""" return '' + special_vars.data_y
            """)
            
    TIMETOOLTIPS = """
            <div style="background-color:#F5F5F5; opacity: 0.95; border: 5px 5px 5px 5px;">
            <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold"> @label
             </span>
             </div>
             <div style = "text-align:left;">"""
    
    if level == "total_imports":
        TIMETOOLTIPS = TIMETOOLTIPS + """
            <span style="font-size: 13px; font-weight: bold"> $data_x{%b %Y}:  $data_y{$0.0a}</span>   
            </div>
            </div>
            """
    if level == "growth":
        TIMETOOLTIPS = TIMETOOLTIPS + """
            <span style="font-size: 13px; font-weight: bold"> $data_x{%b %Y}:  $data_y{0.0}%</span>   
            </div>
            </div>
            """

    p.add_tools(HoverTool(tooltips = TIMETOOLTIPS,  line_policy='nearest', formatters={'$data_x': 'datetime'}))
    p.title.text_font_size = '13pt'
    p.background_fill_color = background 
    p.background_fill_alpha = 0.75
    p.border_fill_color = background 
    
    
    pandemic_box = BoxAnnotation(left=dt.datetime(2020,3,1), right=dt.datetime(2020,12,31), fill_color='red', fill_alpha=0.1)
    p.add_layout(pandemic_box)
    
    #p.yaxis.axis_label = 
    p.yaxis.axis_label_text_font_style = 'bold'
    p.yaxis.axis_label_text_font_size = "13px"

    p.yaxis.minor_tick_line_color = None
    
    if level == "total_imports":
        p.yaxis.formatter = NumeralTickFormatter(format="($0. a)")
        
    if level == "growth":
        p.y_range.end = 300
    
    p.outline_line_color = None
    p.sizing_mode= "scale_both"
    p.max_height = height
    p.max_width = width
    
    p.toolbar.active_drag = None
    p.toolbar.autohide = True
    p.min_border_left = 0
    p.min_border_bottom = 0

    return p

In [53]:
def make_panel_fig():
    
    plevel = make_time_by_product(productdf, "total_imports")
    
    pchange = make_time_by_product(productdf, "growth")
    
    #################################################################33
    
    clevel = make_time_by_country(country_df, "total_imports")
    
    cchange = make_time_by_country(country_df, "growth")
    
    #################################################################
    
    tab1 = Panel(child= plevel, title="Imports in $")

    tab2 = Panel(child= pchange, title="Imports as % YoY")
        
    tab3 = Panel(child= clevel, title="Imports in $")

    tab4 = Panel(child= cchange, title="Imports as % YoY")
        
    outfig = row(Tabs(tabs=[tab4,  tab3], tabs_location = "above"), Tabs(tabs=[tab2,  tab1], tabs_location = "above"))

    return outfig

In [54]:
pchange = make_time_by_product(productdf, "growth")

show(pchange)

In [55]:
clevel = make_time_by_country(country_df, "total_imports")
    
cchange = make_time_by_country(country_df, "growth")

tab3 = Panel(child= clevel, title="Imports in $")

tab4 = Panel(child= cchange, title="Imports as % YoY")

output_file('.\\docs\\' + "us_imports_aul.html")

outfig = Tabs(tabs=[tab3,  tab4], tabs_location = "above")

show(outfig)